Ahora vamos a analizar que variables son las que tienen una mayor importancia a la hora de determinar el tiempo por vuelta.

Primero implementamos el dataset y entrenamos el modelo que hayamos elegido.

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler


In [6]:
# Ruta al archivo CSV
archivo_csv = 'Datasets/240310F1_Dataset.csv'
encoder = LabelEncoder()
# Leer el archivo CSV
df_F1 = pd.read_csv(archivo_csv)


df_F1.drop('DesgasteVuelta', axis=1, inplace=True)
df_F1.drop('Clima', axis=1, inplace=True)
df_F1.drop('TemperaturaPista', axis=1, inplace=True)

encoder = LabelEncoder()
df_F1['Piloto'] = encoder.fit_transform(df_F1['Piloto'])

fuel_discretizer = KBinsDiscretizer(n_bins=20, encode='ordinal', strategy='uniform')
Wheel_life_discretizer = KBinsDiscretizer(n_bins = 6, encode='ordinal', strategy='uniform')
Trust_Discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
#Water_Discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')

#df_F1['CargaCombustible'] = fuel_discretizer.fit_transform(df_F1[['CargaCombustible']])
#df_F1['VidaNeumatico'] = Wheel_life_discretizer.fit_transform(df_F1[['VidaNeumatico']])
df_F1['ConfianzaPilotoCoche'] = Trust_Discretizer.fit_transform(df_F1[['ConfianzaPilotoCoche']])
df_F1['ConfianzaPilotoPista'] = Trust_Discretizer.fit_transform(df_F1[['ConfianzaPilotoPista']])
#df_F1['CantidadAgua'] = Water_Discretizer.fit_transform(df_F1[['CantidadAgua']])

df_F1.dropna(inplace=True)

# Eliminar filas donde 'SafetyCar' es True
df_F1 = df_F1[df_F1['SafetyCar'] == False]
df_F1.drop('SafetyCar', axis=1, inplace=True)


# Eliminar filas donde haya trafico ya que no son representativas.
df_F1 = df_F1[df_F1['Trafico'] == False]
df_F1.drop('Trafico', axis=1, inplace=True)


df_F1 = pd.get_dummies(df_F1, columns = ['Circuito'] )
df_F1 = pd.get_dummies(df_F1, columns = ['Neumáticos'] )
df_F1 = pd.get_dummies(df_F1, columns = ['ModoPilotaje'] )
df_F1 = pd.get_dummies(df_F1, columns = ['GomaEnPista'] )
df_F1 = pd.get_dummies(df_F1, columns = ['AgarreAsfalto'] )



/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


In [7]:

# Selecting the 'CargaCombustible' column
carga_combustible = df_F1['CargaCombustible'].values.reshape(-1, 1)

min_max_scaler = MinMaxScaler()
carga_combustible_minmax = min_max_scaler.fit_transform(carga_combustible)

df_F1['CargaCombustible'] = carga_combustible_minmax


# Selecting the 'VidaNeumatico' column
Vida_neumatico = df_F1['VidaNeumatico'].values.reshape(-1, 1)

vida_neumatico_min_max = min_max_scaler.fit_transform(Vida_neumatico)

df_F1['VidaNeumatico'] = vida_neumatico_min_max


# Selecting the 'CantidadAgua' column
Cantidad_Agua = df_F1['CantidadAgua'].values.reshape(-1, 1)

Cantidad_Agua_min_max = min_max_scaler.fit_transform(Cantidad_Agua)

df_F1['CantidadAgua'] = Cantidad_Agua_min_max


X = df_F1.drop('TiempoVuelta', axis=1)
Y = df_F1[['TiempoVuelta']]

Entrenamos el random forest ya que es el modelo que mejores resultados nos ha dado:

In [3]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
# Crear y entrenar el modelo
model = RandomForestRegressor(n_estimators=200)
model.fit(X, Y)

/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(n_estimators=200)

La siguiente funcion, transforma los datos de entrada en datos preprocesados para ser tratados por el modelo.

In [10]:
def transform_data(Circuit, sesion, Neumaticos, ModoPilotaje, 
            CargaCombustible, GomaEnpista,
            Agarre, Sobrecalentamiento, VidaNeumatico, agua, confpilotopista,
            confpilotocoche, piloto, combustible):
    
    if piloto == "Sainz":
        piloto = 0
    else: piloto = 1


    if Circuit == 'Bahrein':
        Circuito_Barein = True
        Circuito_Jeddah = False
        Circuito_Australia = False
        Circuito_Imola = False
        Circuito_Miami = False
    elif Circuit == 'Jeddah':
        Circuito_Barein = False
        Circuito_Jeddah = True
        Circuito_Australia = False
        Circuito_Imola = False
        Circuito_Miami = False
    elif Circuit == 'Australia':
        Circuito_Barein = False
        Circuito_Jeddah = False
        Circuito_Australia = True
        Circuito_Imola = False
        Circuito_Miami = False
    elif Circuit == 'Imola':
        Circuito_Barein = False
        Circuito_Jeddah = False
        Circuito_Australia = False
        Circuito_Imola = True
        Circuito_Miami = False
    elif Circuit == 'Miami':
        Circuito_Barein = False
        Circuito_Jeddah = False
        Circuito_Australia = False
        Circuito_Imola = False
        Circuito_Miami = True
    

    if Neumaticos == 'Soft':
        Neumaticos_Soft = True
        Neumaticos_Medium = False
        Neumaticos_Hard = False
        Neumaticos_Inter = False
        Neumaticos_Rain = False
    elif Neumaticos == 'Medium':
        Neumaticos_Soft = False
        Neumaticos_Medium = True
        Neumaticos_Hard = False
        Neumaticos_Inter = False
        Neumaticos_Rain = False
    elif Neumaticos == 'Hard':
        Neumaticos_Soft = False
        Neumaticos_Medium = False
        Neumaticos_Hard = True
        Neumaticos_Inter = False
        Neumaticos_Rain = False
    elif Neumaticos == 'Inter':
        Neumaticos_Soft = False
        Neumaticos_Medium = False
        Neumaticos_Hard = False
        Neumaticos_Inter = True
        Neumaticos_Rain = False
    else:
        Neumaticos_Soft = False
        Neumaticos_Medium = False
        Neumaticos_Hard = False
        Neumaticos_Inter = False
        Neumaticos_Rain = True

    if ModoPilotaje == 'Push':
        ModoPilotaje_Push = True
        ModoPilotaje_Balanced = False
        ModoPilotaje_Save = False
    elif ModoPilotaje == 'Balanced':
        ModoPilotaje_Push = False
        ModoPilotaje_Balanced = True
        ModoPilotaje_Save = False
    elif ModoPilotaje == 'Save':
        ModoPilotaje_Push = False
        ModoPilotaje_Balanced = False
        ModoPilotaje_Save = True

    if GomaEnpista == 'VeryLow':
        GomaEnpista_VeryLow = True
        GomaEnpista_Low = False
        GomaEnpista_Average = False
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'Low':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = True
        GomaEnpista_Average = False
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'Average':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = False
        GomaEnpista_Average = True
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'High':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = False
        GomaEnpista_Average = False
        GomaEnpista_High = True
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'VeryHigh':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = False
        GomaEnpista_Average = False
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = True

    if Agarre == 'Low':
        Agarre_Low = True
        Agarre_Average = False
        Agarre_High = False
        Agarre_VeryHigh = False
    elif Agarre == 'Average':
        Agarre_Low = False
        Agarre_Average = True
        Agarre_High = False
        Agarre_VeryHigh = False
    elif Agarre == 'High':
        Agarre_Low = False
        Agarre_Average = False
        Agarre_High = True
        Agarre_VeryHigh = False
    elif Agarre == 'VeryHigh':
        Agarre_Low = False
        Agarre_Average = False
        Agarre_High = False
        Agarre_VeryHigh = True


    new_df = pd.DataFrame({
        
        'Entrenamientos' : [sesion],
        'CargaCombustible' : [CargaCombustible],
        'Sobrecalentamiento' : [Sobrecalentamiento],
        'VidaNeumatico' : [VidaNeumatico],
        'CantidadAgua' : [agua],
        'ConfianzaPilotoPista' : [confpilotopista],
        'ConfianzaPilotoCoche' : [confpilotocoche],
        'Piloto' : [piloto],
        'CombustibleAlto' : [combustible],
        'Circuito_Australia' : [Circuito_Australia],
        'Circuito_Bahrein' : [Circuito_Barein],
        'Circuito_Imola' : [Circuito_Imola],
        'Circuito_Jeddah' : [Circuito_Jeddah],
        'Circuito_Miami' : [Circuito_Miami],
        'Neumáticos_Hard' : [Neumaticos_Hard],
        'Neumáticos_Inter' : [Neumaticos_Inter],
        'Neumáticos_Medium' : [Neumaticos_Medium],
        'Neumáticos_Rain' : [Neumaticos_Rain],
        'Neumáticos_Soft' : [Neumaticos_Soft],
        'ModoPilotaje_Balanced' : [ModoPilotaje_Balanced],
        'ModoPilotaje_Push' : [ModoPilotaje_Push],
        'ModoPilotaje_Save' : [ModoPilotaje_Save],
        'GomaEnPista_Average' : [GomaEnpista_Average],
        'GomaEnPista_High' : [GomaEnpista_High],
        'GomaEnPista_Low' : [GomaEnpista_Low],
        'GomaEnPista_VeryHigh' : [GomaEnpista_VeryHigh],
        'GomaEnPista_VeryLow' : [GomaEnpista_VeryLow],
        'AgarreAsfalto_Average' : [Agarre_Average],
        'AgarreAsfalto_High' : [Agarre_High],
        'AgarreAsfalto_Low' : [Agarre_Low],
        'AgarreAsfalto_VeryHigh' : [Agarre_VeryHigh]
    })

    return new_df

Ahora vamos a ver la influencia de las diferentes variables en el tiempo por vuelta.

Para ello vamos a probar a predecir el tiempo por vuelta con unas caracteristicas por defecto, y alterando únicamente una característica, veremos en que medida altera el tiempo por vuelta.

Vamos a probar cambiando el circuito en el que se corre.

In [11]:
new_df0 = transform_data("Bahrein", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Imola", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Miami", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Australia", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)


In [12]:
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(model.predict(new_df3))

[94.33415]
[77.388265]
[91.912465]
[81.444415]


Ahora vamos a probar a alterar la variable de el tipo de sesión en el que nos encontramos. Si entrenamientos o carrera.

In [13]:
new_df0 = transform_data("Bahrein", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", False, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Imola", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", False, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df4 = transform_data("Jeddah", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Jeddah", False, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)

In [14]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(" ")
print("Circuito 2:")
print(model.predict(new_df2))
print(model.predict(new_df3))
print(" ")
print("Circuito 3:")
print(model.predict(new_df4))
print(model.predict(new_df5))

Circuito 1:
[94.33415]
[96.778305]
 
Circuito 2:
[77.388265]
[77.40255]
 
Circuito 3:
[91.860645]
[93.341885]


Ahora vamos a probar el tipo de neumáticos:

In [57]:
new_df0 = transform_data("Bahrein", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Medium", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Bahrein", True, "Hard", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df4 = transform_data("Imola", True, "Medium", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Imola", True, "Hard", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", True, "Soft", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df7 = transform_data("Jeddah", True, "Medium", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df8 = transform_data("Jeddah", True, "Hard", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)


In [58]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))


Circuito 1:
[94.239045]
[96.15933]
[97.75447]
 
Circuito 2:
[77.390705]
[77.794545]
[78.78913]
 
Circuito 3:
[91.85625]
[92.494845]
[92.744445]


Ahora vamos a ver las diferencias entre los modos de conducción

In [60]:
new_df0 = transform_data("Bahrein", True, "Medium", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Medium", "Balanced", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Bahrein", True, "Medium", "Save", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Medium", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df4 = transform_data("Imola", True, "Medium", "Balanced", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Imola", True, "Medium", "Save", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", True, "Medium", "Push", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df7 = transform_data("Jeddah", True, "Medium", "Balanced", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df8 = transform_data("Jeddah", True, "Medium", "Save", 0.15, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)


In [61]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))

Circuito 1:
[96.15933]
[96.38276]
[96.55733]
 
Circuito 2:
[77.794545]
[78.099705]
[78.23117]
 
Circuito 3:
[92.494845]
[92.82593]
[93.216975]


Ahora vamos a ver la influencia de la carga de combustible

In [66]:
new_df0 = transform_data("Bahrein", True, "Medium", "Balanced", 0.1, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Medium", "Balanced", 0.65, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Bahrein", True, "Medium", "Balanced", 0.95, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Medium", "Balanced", 0.1, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df4 = transform_data("Imola", True, "Medium", "Balanced", 0.65, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Imola", True, "Medium", "Balanced", 0.95, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", True, "Medium", "Balanced", 0.1, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df7 = transform_data("Jeddah", True, "Medium", "Balanced", 0.65, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df8 = transform_data("Jeddah", True, "Medium", "Balanced", 0.95, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)

In [67]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))

Circuito 1:
[96.47286]
[96.512935]
[96.96189]
 
Circuito 2:
[78.117225]
[80.47663]
[80.944785]
 
Circuito 3:
[92.82586]
[94.00341]
[95.34407]


Ahora vamos a probar la goma en pista:

In [70]:
new_df0 = transform_data("Bahrein", True, "Medium", "Balanced", 0.5, "VeryLow", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Medium", "Balanced", 0.5, "Average", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Bahrein", True, "Medium", "Balanced", 0.5, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Medium", "Balanced", 0.5, "VeryLow", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df4 = transform_data("Imola", True, "Medium", "Balanced", 0.5, "Average", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Imola", True, "Medium", "Balanced", 0.5, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", True, "Medium", "Balanced", 0.5, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df7 = transform_data("Jeddah", True, "Medium", "Balanced", 0.5, "Average", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df8 = transform_data("Jeddah", True, "Medium", "Balanced", 0.5, "VeryHigh", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)

In [71]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))

Circuito 1:
[96.04319]
[95.90045]
[96.10854]
 
Circuito 2:
[78.495005]
[78.49894]
[78.495005]
 
Circuito 3:
[93.063005]
[92.934365]
[93.063005]


Ahora vamos a probar el agarre en pista.

In [111]:
new_df0 = transform_data("Bahrein", True, "Medium", "Balanced", 0.5, "Average", "Low", False, 0.745, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Medium", "Balanced", 0.5, "Average", "Average", False, 0.745, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Bahrein", True, "Medium", "Balanced", 0.5, "Average", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Medium", "Balanced", 0.5, "Average", "Low", False, 0.745, 0, 2, 2, "Sainz", True)
new_df4 = transform_data("Imola", True, "Medium", "Balanced", 0.5, "Average", "Average", False, 0.745, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Imola", True, "Medium", "Balanced", 0.5, "Average", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", True, "Medium", "Balanced", 0.5, "Average", "Low", False, 0.745, 0, 2, 2, "Sainz", True)
new_df7 = transform_data("Jeddah", True, "Medium", "Balanced", 0.5, "Average", "Average", False, 0.745, 0, 2, 2, "Sainz", True)
new_df8 = transform_data("Jeddah", True, "Medium", "Balanced", 0.5, "Average", "VeryHigh", False, 0.745, 0, 2, 2, "Sainz", True)

In [112]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))

Circuito 1:
[95.81634]
[95.835775]
[95.90045]
 
Circuito 2:
[78.49894]
[78.50682]
[78.49894]
 
Circuito 3:
[92.92276]
[92.9231]
[92.934365]


Ahora vamos a probar la vida del neumático:

In [149]:
new_df0 = transform_data("Bahrein", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.95, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.45, 0, 2, 2, "Sainz", True)
new_df2 = transform_data("Bahrein", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.05, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.95, 0, 2, 2, "Sainz", True)
new_df4 = transform_data("Imola", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.45, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Imola", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.05, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.95, 0, 2, 2, "Sainz", True)
new_df7 = transform_data("Jeddah", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.45, 0, 2, 2, "Sainz", True)
new_df8 = transform_data("Jeddah", False, "Soft", "Push", 0.5, "Average", "Average", False, 0.05, 0, 2, 2, "Sainz", True)

In [150]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))

Circuito 1:
[97.523435]
[97.969875]
[98.69513]
 
Circuito 2:
[77.603775]
[77.676385]
[77.81432]
 
Circuito 3:
[94.64538]
[95.08465]
[95.078495]


Ahora vamos a probar la confianza en la pista:

In [98]:
new_df0 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 0, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 1, 2, "Sainz", True)
new_df2 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 0, 2, "Sainz", True)
new_df4 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 1, 2, "Sainz", True)
new_df5 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 0, 2, "Sainz", True)
new_df7 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 1, 2, "Sainz", True)
new_df8 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)

In [99]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))

Circuito 1:
[95.011415]
[95.01602]
[94.917175]
 
Circuito 2:
[77.49228]
[77.49228]
[77.51655]
 
Circuito 3:
[92.180335]
[92.13225]
[92.13523]


Ahora vamos a probar la confianza en el coche:

In [151]:
new_df0 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 0, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 1, "Sainz", True)
new_df2 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 0, "Sainz", True)
new_df4 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 1, "Sainz", True)
new_df5 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df6 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 0, "Sainz", True)
new_df7 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 1, "Sainz", True)
new_df8 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)

In [101]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(model.predict(new_df2))
print(" ")
print("Circuito 2:")
print(model.predict(new_df3))
print(model.predict(new_df4))
print(model.predict(new_df5))
print(" ")
print("Circuito 3:")
print(model.predict(new_df6))
print(model.predict(new_df7))
print(model.predict(new_df8))

Circuito 1:
[94.988905]
[94.988905]
[94.917175]
 
Circuito 2:
[77.51655]
[77.51655]
[77.51655]
 
Circuito 3:
[92.185115]
[92.13102]
[92.13523]


Ahora vamos a probar el piloto en el coche:

In [107]:
new_df0 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Leclerc", True)
new_df2 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Leclerc", True)
new_df4 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Leclerc", True)

In [108]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(" ")
print("Circuito 2:")
print(model.predict(new_df2))
print(model.predict(new_df3))
print(" ")
print("Circuito 3:")
print(model.predict(new_df4))
print(model.predict(new_df5))

Circuito 1:
[94.917175]
[95.010335]
 
Circuito 2:
[77.51655]
[77.587475]
 
Circuito 3:
[92.13523]
[92.1793]


Ahora vamos a probar el modo de gasto del combustible:

In [109]:
new_df0 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df1 = transform_data("Bahrein", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", False)
new_df2 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df3 = transform_data("Imola", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", False)
new_df4 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", True)
new_df5 = transform_data("Jeddah", True, "Soft", "Push", 0.5, "Average", "Average", False, 0.5, 0, 2, 2, "Sainz", False)

In [110]:
print("Circuito 1:")
print(model.predict(new_df0))
print(model.predict(new_df1))
print(" ")
print("Circuito 2:")
print(model.predict(new_df2))
print(model.predict(new_df3))
print(" ")
print("Circuito 3:")
print(model.predict(new_df4))
print(model.predict(new_df5))

Circuito 1:
[94.917175]
[95.007595]
 
Circuito 2:
[77.51655]
[77.552535]
 
Circuito 3:
[92.13523]
[92.30469]


Despues de haber visto que variables influyen mas en el tiempo por vuelta, vamos a probar a entrenar el modelo solo con variables que influyen realmente en el tiempo por vuelta.

In [10]:
# Ruta al archivo CSV
archivo_csv = 'Datasets/240310F1_Dataset.csv'
encoder = LabelEncoder()
# Leer el archivo CSV
df_F1 = pd.read_csv(archivo_csv)


df_F1.drop('DesgasteVuelta', axis=1, inplace=True)
df_F1.drop('Clima', axis=1, inplace=True)
df_F1.drop('TemperaturaPista', axis=1, inplace=True)
df_F1.drop( 'ConfianzaPilotoCoche', axis=1, inplace=True)
df_F1.drop( 'ConfianzaPilotoPista', axis=1, inplace=True)
df_F1.drop( 'AgarreAsfalto', axis=1, inplace=True)
df_F1.drop( 'GomaEnPista', axis=1, inplace=True)


#Eliminamos las filas que contengan NaN
df_F1.dropna(inplace=True)

# Eliminar filas donde 'SafetyCar' es True
df_F1 = df_F1[df_F1['SafetyCar'] == False]
df_F1.drop('SafetyCar', axis=1, inplace=True)


# Eliminar filas donde haya trafico ya que no son representativas.
df_F1 = df_F1[df_F1['Trafico'] == False]
df_F1.drop('Trafico', axis=1, inplace=True)

encoder = LabelEncoder()
df_F1['Piloto'] = encoder.fit_transform(df_F1['Piloto'])

df_F1 = pd.get_dummies(df_F1, columns = ['Circuito'] )
df_F1 = pd.get_dummies(df_F1, columns = ['Neumáticos'] )
df_F1 = pd.get_dummies(df_F1, columns = ['ModoPilotaje'] )


In [11]:

# Selecting the 'CargaCombustible' column
carga_combustible = df_F1['CargaCombustible'].values.reshape(-1, 1)

min_max_scaler = MinMaxScaler()
carga_combustible_minmax = min_max_scaler.fit_transform(carga_combustible)

df_F1['CargaCombustible'] = carga_combustible_minmax


# Selecting the 'VidaNeumatico' column
Vida_neumatico = df_F1['VidaNeumatico'].values.reshape(-1, 1)

vida_neumatico_min_max = min_max_scaler.fit_transform(Vida_neumatico)

df_F1['VidaNeumatico'] = vida_neumatico_min_max


# Selecting the 'CantidadAgua' column
Cantidad_Agua = df_F1['CantidadAgua'].values.reshape(-1, 1)

Cantidad_Agua_min_max = min_max_scaler.fit_transform(Cantidad_Agua)

df_F1['CantidadAgua'] = Cantidad_Agua_min_max


X = df_F1.drop('TiempoVuelta', axis=1)
print(X)
Y = df_F1[['TiempoVuelta']]

      Entrenamientos  CargaCombustible  Sobrecalentamiento  VidaNeumatico  \
0               True          0.406827               False       0.955224   
1               True          0.320111               False       0.955224   
2               True          0.389299               False       0.895522   
3               True          0.301661               False       0.895522   
4               True          0.372694               False       0.835821   
...              ...               ...                 ...            ...   
1523           False          0.036900               False       0.119403   
1524           False          0.014760               False       0.208955   
1525           False          0.019373               False       0.059701   
1526           False          0.000000               False       0.149254   
1527           False          0.004613               False       0.000000   

      CantidadAgua  Piloto  CombustibleAlto  Circuito_Australia  \
0       

In [12]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [13]:
# Supongamos que df es tu DataFrame y que 'target' es la columna objetivo
df = df_F1 # Reemplaza con tu DataFrame
X = df.drop('TiempoVuelta', axis=1)  # Reemplaza 'target' con el nombre de tu columna objetivo
y = df['TiempoVuelta']  # De nuevo, reemplaza 'target' con el nombre de tu columna objetivo
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = RandomForestRegressor(n_estimators=200)
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

Error Cuadrático Medio Promedio: 0.5329343627846699
Error Absoluto Medio Promedio: 0.24746593181818635


El error es prácticamente el mismo y hemos conseguido eliminar de los datos de entrada para entrenar al modelo 11 columnas.
Por lo que el modelo será mas eficiente que el que teniamos anteriormente.

Ahora vamos a comprobar si creando un modelo para cada circuito da mejores, iguales o peores resultados que los que tenemos.

In [14]:
# Ruta al archivo CSV
archivo_csv = 'Datasets/240310F1_Dataset.csv'
encoder = LabelEncoder()
# Leer el archivo CSV
df_F1 = pd.read_csv(archivo_csv)


df_F1.drop('DesgasteVuelta', axis=1, inplace=True)
df_F1.drop('Clima', axis=1, inplace=True)
df_F1.drop('TemperaturaPista', axis=1, inplace=True)
df_F1.drop( 'ConfianzaPilotoCoche', axis=1, inplace=True)
df_F1.drop( 'ConfianzaPilotoPista', axis=1, inplace=True)
df_F1.drop( 'AgarreAsfalto', axis=1, inplace=True)
df_F1.drop( 'GomaEnPista', axis=1, inplace=True)


#Eliminamos las filas que contengan NaN
df_F1.dropna(inplace=True)

# Eliminar filas donde 'SafetyCar' es True
df_F1 = df_F1[df_F1['SafetyCar'] == False]
df_F1.drop('SafetyCar', axis=1, inplace=True)


# Eliminar filas donde haya trafico ya que no son representativas.
df_F1 = df_F1[df_F1['Trafico'] == False]
df_F1.drop('Trafico', axis=1, inplace=True)

encoder = LabelEncoder()
df_F1['Piloto'] = encoder.fit_transform(df_F1['Piloto'])

df_F1 = pd.get_dummies(df_F1, columns = ['Neumáticos'] )
df_F1 = pd.get_dummies(df_F1, columns = ['ModoPilotaje'] )


In [15]:

# Selecting the 'CargaCombustible' column
carga_combustible = df_F1['CargaCombustible'].values.reshape(-1, 1)

min_max_scaler = MinMaxScaler()
carga_combustible_minmax = min_max_scaler.fit_transform(carga_combustible)

df_F1['CargaCombustible'] = carga_combustible_minmax


# Selecting the 'VidaNeumatico' column
Vida_neumatico = df_F1['VidaNeumatico'].values.reshape(-1, 1)

vida_neumatico_min_max = min_max_scaler.fit_transform(Vida_neumatico)

df_F1['VidaNeumatico'] = vida_neumatico_min_max


# Selecting the 'CantidadAgua' column
Cantidad_Agua = df_F1['CantidadAgua'].values.reshape(-1, 1)

Cantidad_Agua_min_max = min_max_scaler.fit_transform(Cantidad_Agua)

df_F1['CantidadAgua'] = Cantidad_Agua_min_max

df_Bahrein = df_F1[df_F1['Circuito'] == 'Bahrein']
df_Bahrein = df_Bahrein.drop(columns=['Circuito'], axis = 1)

df_Jeddah = df_F1[df_F1['Circuito'] == 'Jeddah']
df_Jeddah = df_Jeddah.drop(columns=['Circuito'], axis = 1)

df_Australia = df_F1[df_F1['Circuito'] == 'Australia']
df_Australia = df_Australia.drop(columns=['Circuito'], axis=1)

df_Imola = df_F1[df_F1['Circuito'] == 'Imola']
df_Imola = df_Imola.drop(columns=['Circuito'], axis=1)

df_Miami = df_F1[df_F1['Circuito'] == 'Miami']
df_Miami = df_Miami.drop('Circuito', axis=1)



In [20]:
# Supongamos que df es tu DataFrame y que 'target' es la columna objetivo
df = df_Miami # Reemplaza con tu DataFrame
X = df.drop('TiempoVuelta', axis=1)  # Reemplaza 'target' con el nombre de tu columna objetivo
y = df['TiempoVuelta']  # De nuevo, reemplaza 'target' con el nombre de tu columna objetivo
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = RandomForestRegressor(n_estimators=200)
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

Error Cuadrático Medio Promedio: 0.08223492818520232
Error Absoluto Medio Promedio: 0.16829617435065852
